Reproducible steps for getting 10-k filings from [EDGAR](https://www.sec.gov/edgar).

You will need the latest copy of the modules.

1. [sec-downloader](https://github.com/Elijas/sec-downloader)
2. [sec-parser](https://github.com/alphanome-ai/sec-parser)


In [ ]:
!pip install sec-downloader
!pip install sec-parser